In [1]:
import glob
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
!pip install funcy
import funcy
from funcy import print_durations
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from torch.cuda.amp import GradScaler
torch.backends.cudnn.benchmark = True
import datetime
import gc
import pickle

/opt/conda/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Defaulting to user installation because normal site-packages is not writeable


In [2]:
start_date = '1980-01-01'

In [3]:
returns_df = pd.read_csv('returns.csv',index_col=0).loc[start_date:]
# returns_df = returns_df.dropna(axis=0,thresh=500).dropna(axis=1,thresh=500)
equity_idx = returns_df.columns
date_idx = returns_df.index
returns_df.clip(returns_df.quantile(0.01,axis=1),returns_df.quantile(0.99,axis=1),axis=0)
returns_df

,0081A,0091A,0099A,0127A,0138A,0146A,0153A,0183B,0223B,0230B,...,ZVXI,ZWRK,ZWS,ZXAIY,ZY,ZYME,ZYNE,ZYXI,ZZ,ZZ1
datadate,,,,,,,,,,,,,,,,,,,,,
1980-01-31,NaN,0.012579,NaN,0.068182,0.039548,NaN,NaN,NaN,0.007752,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.137500
1980-02-29,NaN,-0.006211,NaN,0.000000,NaN,NaN,NaN,NaN,-0.086462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.164835
1980-03-31,NaN,-0.306250,NaN,0.007092,NaN,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.118421
1980-04-30,NaN,0.252252,NaN,-0.126761,NaN,NaN,NaN,NaN,0.139130,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.029851
1980-05-31,NaN,0.071942,NaN,0.177419,NaN,NaN,NaN,NaN,0.036336,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.123077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.001025,-0.033076,0.001111,-0.145283,-0.096180,-0.096606,0.021548,NaN,NaN
2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.001026,0.038516,0.054384,-0.261589,-0.184983,-0.167630,-0.221094,NaN,NaN
2022-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.004620,-0.160989,0.084211,-0.222720,-0.456986,-0.093750,-0.115145,NaN,NaN


In [4]:
feat_comb_df = pd.read_csv('features_combined.csv').set_index(['tic','rdq']).drop(['FYEARQ','FQTR'],axis=1)
feat_comb_df = feat_comb_df[sorted(feat_comb_df.columns)]
feat_comb_df

A2ME  AC       ATO  BEME         C  CF  CF2P       CTO  \
tic   rdq                                                                      
0015B 1984-01-16  0.381941 NaN       NaN   NaN  0.214994 NaN   NaN       NaN   
      1984-03-15  0.557109 NaN       NaN   NaN  0.274170 NaN   NaN       NaN   
      1984-05-25  0.590922 NaN       NaN   NaN  0.140065 NaN   NaN       NaN   
      1984-08-27  0.651735 NaN       NaN   NaN  0.009504 NaN   NaN       NaN   
      1985-01-01  0.871765 NaN  0.760755   NaN  0.019574 NaN   NaN  0.778316   
...                    ...  ..       ...   ...       ...  ..   ...       ...   
ZZZ1  2000-08-14  1.171566 NaN  0.073420   NaN  0.036884 NaN   NaN  0.067789   
      2000-11-13  1.185838 NaN  0.079706   NaN  0.029840 NaN   NaN  0.069810   
      2001-03-28       NaN NaN  0.075438   NaN  0.041668 NaN   NaN  0.065717   
      2001-05-14       NaN NaN  0.085226   NaN  0.029079 NaN   NaN  0.082268   
      2001-08-13       NaN NaN  0.091314   NaN  0.040394 NaN   NaN  0.085981   

                       D2A       D2P  ...  OP       PCM        PM  PROF   Q  \
tic   rdq                             ...                                     
0015B 1984-01-16       NaN       NaN  ... NaN       NaN       NaN   NaN NaN   
      1984-03-15       NaN       NaN  ... NaN       NaN       NaN   NaN NaN   
      1984-05-25       NaN       NaN  ... NaN       NaN       NaN   NaN NaN   
      1984-08-27  0.015213  0.000000  ... NaN -0.968086  0.247569   NaN NaN   
      1985-01-01  0.012021  0.000000  ... NaN -0.950109  0.230870   NaN NaN   
...                    ...       ...  ...  ..       ...       ...   ...  ..   
ZZZ1  2000-08-14       NaN  0.006326  ... NaN  0.523240  1.000000   NaN NaN   
      2000-11-13       NaN  0.004737  ... NaN  0.559099  1.000000   NaN NaN   
      2001-03-28       NaN       NaN  ... NaN  0.540736  1.000000   NaN NaN   
      2001-05-14       NaN       NaN  ... NaN  0.622717  1.000000   NaN NaN   
      2001-08-13       NaN       NaN  ... NaN  0.653604  1.000000   NaN NaN   

                       RNA       ROA  ROE       S2P     SGA2S  
tic   rdq                                                      
0015B 1984-01-16       NaN       NaN  NaN       NaN       NaN  
      1984-03-15       NaN       NaN  NaN       NaN       NaN  
      1984-05-25       NaN       NaN  NaN       NaN       NaN  
      1984-08-27       NaN       NaN  NaN  1.418175  0.731681  
      1985-01-01  0.175636  0.125346  NaN  1.499765  0.748516  
...                    ...       ...  ...       ...       ...  
ZZZ1  2000-08-14  0.073420  0.018000  NaN  0.098857       NaN  
      2000-11-13  0.079706  0.021708  NaN  0.104211       NaN  
      2001-03-28  0.075438  0.021779  NaN       NaN       NaN  
      2001-05-14  0.085226  0.044324  NaN       NaN       NaN  
      2001-08-13  0.091314  0.062878  NaN       NaN       NaN  

[924199 rows x 29 columns]

In [5]:
MIN_AVAILABLE_FEATURES = feat_comb_df.shape[1]//2

full_df_lst = []
for date in tqdm(date_idx):
    sub = feat_comb_df.loc[feat_comb_df.index.get_level_values(1) < date]
    sub = sub.groupby(sub.index.get_level_values('tic')).last(1)
    sub = sub.transpose()
    
    sub = sub.rank(pct=True,axis=1)*2-1
    idx = (sub.isna().sum(axis=0) >= MIN_AVAILABLE_FEATURES).index
    sub = sub[idx].fillna(0)
    
#     sub = sub.fillna(sub.median(axis=1),axis=0)
    
    sub['date'] = date
    sub =  sub.set_index(['date',sub.index])
    
    full_df_lst.append(sub)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 507/507 [02:01<00:00,  4.19it/s]


In [6]:
gc.collect()

0

# Train Test Splits

In [7]:
orig = (48,360)
valid = (360,420)
test = (420,505)
# Train: 1984 to 2010
# Valid: 2010 to 2015
# Test: 2015 to 2022

In [8]:
class AEDataSet(torch.utils.data.Dataset):
    def __init__(self, full_df_lst, returns_df, date_idx, start_i, end_i, rhs_input_assets=False,lag=1):
        self.full_df_lst = full_df_lst
        self.returns_df = returns_df
        self.date_idx = date_idx
        self.start_i, self.end_i = start_i, end_i
        self.rhs_input_assets = rhs_input_assets
        self.epsilon=1e-5
        self.lag=lag
        
        self.output_date_idx = date_idx[start_i+lag:end_i]
        
    def __len__(self):
        return self.end_i - self.start_i - self.lag
    def __getitem__(self, idx):
        input_idx = idx
        idx += self.start_i+self.lag
        date = self.date_idx[idx] # returns date
        
        mask = ~np.isnan(self.returns_df.loc[date].to_numpy())
        traded_assets_idx = self.returns_df.columns[mask]
        
        lhs = self.full_df_lst[idx - self.lag]
        lhs_idx = lhs.columns
        
        comb_idx = traded_assets_idx.intersection(lhs_idx)
        
        lhs = lhs.reindex(comb_idx,axis=1,fill_value=0).to_numpy()
        
        rhs_tmp = self.returns_df.loc[date,comb_idx]
#         rhs_tmp = rhs_tmp.clip(rhs_tmp.quantile(0.01),rhs_tmp.quantile(0.99))
        output = rhs_tmp.to_numpy()

        lhs_tmp = np.concatenate([lhs,np.expand_dims(np.ones(lhs.shape[1]),0)],axis=0)
        
        
        if self.rhs_input_assets:
            rhs = output
        else:
            try:
                rhs = (np.linalg.inv(lhs_tmp @ lhs_tmp.T) @ lhs_tmp) @ rhs_tmp
            except Exception as e:
                print("idx:",input_idx)
                print(e)
                raise e
        return lhs,rhs,output,comb_idx
        
        
def collate_fn(batch):

    len_x = [len(output) for lhs,rhs,output,_ in batch]
    full_size = max(len_x)
    
    batch_lhs = [nn.functional.pad(torch.Tensor(lhs),pad=(0,full_size-lhs.shape[1],0,0)) for lhs,rhs,output,_ in batch]
    batch_rhs = [torch.Tensor(rhs) for lhs,rhs,output,_ in batch]
    batch_output = [nn.functional.pad(torch.Tensor(output),pad=(0,full_size-len(output))) for lhs,rhs,output,_ in batch]
    batch_assets = [idx for lhs,rhs,output,idx in batch]
    
    return torch.cat([x.unsqueeze(0) for x in batch_lhs]),\
        torch.cat([x.unsqueeze(0) for x in batch_rhs]),\
        torch.cat([x.unsqueeze(0) for x in batch_output]), torch.tensor(len_x), batch_assets

# Conditional Autoencoder

In [9]:
class ConditionalAutoEncoder(nn.Module):
    def __init__(self, n_alphas, n_factors, n_hidden_lhs, n_hidden_rhs, 
                    batch_norm_lhs=True, batch_norm_rhs=False,
                    dropout_p=0, initialization=True, rhs_input_assets = False, rhs_activation = True,
                    bn_momentum=0.1, bn_track_running_stats = True):
        # n_assets can be assets or portfolios:
            # In both cases, we are interested in asset return or portfolio return on the RHS
            # LHS is the original factor 
        super(ConditionalAutoEncoder, self).__init__()

        self.n_alphas = n_alphas
        self.n_factors = n_factors
        self.n_hidden_lhs = list(n_hidden_lhs)
        self.n_hidden_rhs = list(n_hidden_rhs)

        self.lhs_lst = []

        prev_lhs_layer_size = self.n_alphas
        layer_lst = []
        for i, v in enumerate(list(n_hidden_lhs)+[n_factors]):
            if batch_norm_lhs and i<len(n_hidden_lhs):
                # layer_lst.append(nn.BatchNorm1d(prev_lhs_layer_size,momentum=bn_momentum,
                #         track_running_stats=bn_track_running_stats))
                layer_lst.append(nn.LayerNorm(prev_lhs_layer_size))
            if dropout_p:
                layer_lst.append(nn.Dropout(dropout_p))
            layer_lst.append(nn.Linear(prev_lhs_layer_size, v, bias=True))
            if initialization:
                nn.init.kaiming_normal_(layer_lst[-1].weight,mode='fan_in',nonlinearity='linear')
            if i<len(n_hidden_lhs):
                layer_lst.append(nn.SELU())
            prev_lhs_layer_size = v
#         layer_lst = layer_lst[:-1]
        # if batch_norm_lhs:
        #     layer_lst.append(nn.BatchNorm1d(prev_lhs_layer_size,momentum=bn_momentum,
        #             track_running_stats=bn_track_running_stats))
        #     # layer_lst.append(nn.LayerNorm(prev_lhs_layer_size))
        # if dropout_p:
        #     layer_lst.append(nn.Dropout(dropout_p))
        # layer_lst.append(nn.Linear(prev_lhs_layer_size, self.n_factors, bias=True))
        # if initialization:
        #     nn.init.xavier_normal_(layer_lst[-1].weight)
        self.lhs = nn.Sequential(*layer_lst)

        # self.lhs_lst = nn.ModuleList(self.lhs_lst)

        rhs_lst = []
#         if rhs_input_assets:
#             if batch_norm_rhs:
#                 # rhs_lst.append(nn.BatchNorm1d(self.n_assets,momentum=bn_momentum,
#                 #     track_running_stats=bn_track_running_stats))
#                 rhs_lst.append(nn.LayerNorm(self.n_assets))
#             if dropout_p:
#                 rhs_lst.append(nn.Dropout(dropout_p))
#             rhs_lst.append(nn.Linear(n_assets,n_alphas+1,bias=True))
#             if initialization:
#                 nn.init.kaiming_normal_(rhs_lst[-1].weight,mode='fan_in',nonlinearity='linear')
#             if rhs_activation:
#                 rhs_lst.append(nn.SELU())

        prev_rhs_layer_size = self.n_alphas+1
        for v in n_hidden_rhs+[n_factors]:
            if batch_norm_rhs:
              # rhs_lst.append(nn.BatchNorm1d(prev_rhs_layer_size,momentum=bn_momentum,
              #         track_running_stats=bn_track_running_stats))
                rhs_lst.append(nn.LayerNorm(prev_rhs_layer_size))
            if dropout_p:
                rhs_lst.append(nn.Dropout(dropout_p))
            rhs_lst.append(nn.Linear(prev_rhs_layer_size,v,bias=True))
            if initialization:
                nn.init.kaiming_normal_(rhs_lst[-1].weight,mode='fan_in',nonlinearity='linear')
            if rhs_activation:
                rhs_lst.append(nn.SELU())
            prev_rhs_layer_size = v
        
        self.rhs = nn.Sequential(*rhs_lst)

#         self.final = nn.Linear(n_assets,n_assets,bias=True)

    def forward(self, x_lhs, x_rhs):
        # idx = 0
        y_lhs_lst = []
        for i in range(x_lhs.shape[2]):
            # y_cur = self.lhs_lst[i](x_lhs[:,idx:idx+self.n_alphas])
            y_cur = self.lhs(x_lhs[:,:,i])
            y_cur = torch.unsqueeze(y_cur,2)
            y_lhs_lst.append(y_cur)
            # idx += self.n_alphas
        y_lhs = torch.cat(y_lhs_lst,dim=2)

        y_rhs = self.rhs(x_rhs).unsqueeze(dim=2)
        self.last_rhs_factors = y_rhs
        out = torch.bmm(y_lhs.transpose(1,2), y_rhs).squeeze(2)
#         out = self.final(out)
        return out, y_rhs

    def forward_given_factors(self, x_lhs, factors):
        y_lhs_lst = []
        for i in range(x_lhs.shape[2]):
            # y_cur = self.lhs_lst[i](x_lhs[:,idx:idx+self.n_alphas])
            y_cur = self.lhs(x_lhs[:,:,i])
            y_cur = torch.unsqueeze(y_cur,2)
            y_lhs_lst.append(y_cur)
            # idx += self.n_alphas
        y_lhs = torch.cat(y_lhs_lst,dim=2)

        y_rhs = factors
        out = torch.bmm(y_lhs.transpose(1,2), y_rhs).squeeze(2)
#         out = self.final(out)
        return out

# Hyperparameter Tuning

In [10]:
from sklearn.metrics import mean_squared_error
@print_durations()
def test(args,model,device,test_loader,criterion):
    model.eval()
        
    output_lst = []
    data_lst = []

    for data_lhs, data_rhs, data_output, data_x_lens, data_assets in test_loader:
        # print("Shapes:",data_lhs.shape, data_rhs.shape, data_output.shape, data_mask.shape)
        data_lhs = data_lhs.float().to(device)
        data_rhs = data_rhs.float().to(device)
        data_output = data_output.float().to(device)
        data_x_lens = data_x_lens.int().to(device)
        output,_ = model(x_lhs = data_lhs, x_rhs = data_rhs)

        data_mask = torch.arange(data_lhs.shape[2])[None, :].to(device) < data_x_lens[:, None]
        output_masked = torch.masked_select(output,data_mask)
        data_output_masked = torch.masked_select(data_output,data_mask)
        assert (len(output_masked) == torch.sum(data_x_lens))
        # print("Loss Inside:",criterion(output_masked,data_output_masked))

        output_lst.append(output_masked)
        data_lst.append(data_output_masked)
    
    full_output = torch.cat(output_lst,axis=0)
    full_data = torch.cat(data_lst,axis=0)

    return mean_squared_error(full_data.cpu().detach().numpy(), full_output.cpu().detach().numpy())

@print_durations()
def train(args, model, device, train_loader, optimizer, criterion, epoch, max_norm=None):
    model.train()
    
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    
    total_loss = 0
    total_reg_loss=0
    
    N = len(train_loader)
    
    for batch_idx, (data_lhs, data_rhs, data_output, data_x_lens, data_assets) in enumerate(train_loader):

        optimizer.zero_grad()
        
        data_lhs = data_lhs.float().to(device)
        data_rhs = data_rhs.float().to(device)
        data_output = data_output.float().to(device)
        data_x_lens = data_x_lens.int().to(device)

        if max_norm: torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        output,_ = model(x_lhs = data_lhs, x_rhs = data_rhs)
        
        data_mask = torch.arange(data_lhs.shape[2])[None, :].to(device) < data_x_lens[:, None]

        output_masked = torch.masked_select(output,data_mask)
        data_output_masked = torch.masked_select(data_output,data_mask)

        loss_tmp = criterion(output_masked, data_output_masked)

        regularization_loss = 0
        for p in model.parameters():
            regularization_loss += torch.sum(abs(p))

        loss = loss_tmp + args['l1_lambda'] * regularization_loss
        
        loss.backward()
        total_loss += loss
        total_reg_loss += regularization_loss * args['l1_lambda'] 

        # print("Gradient:", loss.grad)
        optimizer.step()

        batch_bar.set_postfix(
            total_loss="{:.04f}".format(float(total_loss / (batch_idx + 1))),
            reg_loss = "{:.04f}".format(float(total_reg_loss / (batch_idx + 1))),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        batch_bar.update()
        
    print(f"Epoch {epoch}, total_loss={float(total_loss / N):.04f}, reg_loss = {float(total_reg_loss / N):.04f}")

In [11]:
# train_data = AEDataSet(full_df_lst,returns_df,date_idx,48,360)
# valid_data = AEDataSet(full_df_lst,returns_df,date_idx,360,420)
# test_data= AEDataSet(full_df_lst,returns_df,date_idx,420,505)

def run(args, n_alphas, full_df_lst, returns_df, date_idx, train_start,
        train_end, valid_start, valid_end, n_window=36, batch_size=16, checkpoint_name=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ConditionalAutoEncoder(
        n_alphas = n_alphas,
        n_factors = args['n_factors'],
        n_hidden_lhs = args['n_hidden_lhs'],
        n_hidden_rhs = args['n_hidden_rhs'],
        batch_norm_lhs = args['batch_norm_lhs'],
        batch_norm_rhs = args['batch_norm_rhs'],
        dropout_p=args['dropout_p'],
        initialization=args['initialization'],
        rhs_input_assets=args['rhs_input_assets'],
        rhs_activation = args['rhs_activation']
    #     bn_track_running_stats=args['bn_track_running_stats'],
    #     bn_momentum=args['bn_momentum']
    )
    
    print(model)
    
    gc.collect()
    torch.cuda.empty_cache()
    
    train_data = AEDataSet(full_df_lst,returns_df,date_idx,train_start,train_end)
    valid_data = AEDataSet(full_df_lst,returns_df,date_idx,valid_start,valid_end)
    extended_valid_data = AEDataSet(full_df_lst,returns_df,date_idx,valid_start-n_window,valid_end)

    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size = batch_size,
        pin_memory=True,
        num_workers=0,
        shuffle=True,
        collate_fn = collate_fn
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_data,
        batch_size = batch_size,
        pin_memory=True,
        num_workers=0,
        shuffle=False,
        collate_fn = collate_fn
    )
    
    extended_valid_loader = torch.utils.data.DataLoader(
        extended_valid_data,
        batch_size = batch_size,
        pin_memory=True,
        num_workers=0,
        shuffle=False,
        collate_fn = collate_fn
    )
    
    
    

    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=args['lr'], weight_decay=args['l2_lambda'])
    criterion = torch.nn.MSELoss()

    train_mse = test(args,model,device,train_loader,criterion)
    valid_mse = test(args,model,device,valid_loader,criterion)
    train_mse_lst, valid_mse_lst = [train_mse], [valid_mse]

    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=args['gamma'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',0.5,patience=5,threshold=1e-3,cooldown=10)
    print("Pre-Train Train MSE:", train_mse)
    print("Pre-Train Valid MSE:", valid_mse)

    best_model = None
    for epoch in range(1, args['epoch']+1):
        log_dict = {}

        gc.collect()
        torch.cuda.empty_cache()

        train(args,model,device,train_loader,optimizer,criterion,epoch,max_norm=args['max_norm'])
        gc.collect()
        torch.cuda.empty_cache()

        log_dict['train_mse'] = test(args,model,device,train_loader,criterion)
        log_dict['valid_mse'] = test(args,model,device,valid_loader,criterion)
        log_dict['lr'] = scheduler.optimizer.state_dict()['param_groups'][0]['lr']

        gc.collect()
        torch.cuda.empty_cache()

        print("Train MSE:",log_dict['train_mse'])
        print("Valid MSE:",log_dict['valid_mse'])

        scheduler.step(log_dict['valid_mse'])
        print('Next Learning Rate:', scheduler.optimizer.state_dict()['param_groups'][0]['lr'])

        print(train_mse_lst)
        print(valid_mse_lst)
        if log_dict['valid_mse'] < np.min(valid_mse_lst):
            best_model = model
            torch.save(best_model, './best_model.model')
            # torch.save(model.state_dict(), args['MODEL_PATH']+"/model_epoch_"+str(epoch))

        train_mse_lst.append(log_dict['train_mse'])
        valid_mse_lst.append(log_dict['valid_mse'])

        if len(valid_mse_lst)>args['patience'] and log_dict['valid_mse'] > \
            np.min(valid_mse_lst[:-args['patience']]):
                break
    
    if checkpoint_name:
        torch.save({
#           'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
#           'scheduler_state_dict': scheduler.state_dict()
          }, f'{checkpoint_name}.checkpoint')
    
    print("Training done. Evaluating...")
    
    output_lst = []
    data_lst = []
    factors_lst = []

    for data_lhs, data_rhs, data_output, data_x_lens, data_assets in tqdm(extended_valid_loader):
        # print("Shapes:",data_lhs.shape, data_rhs.shape, data_output.shape, data_mask.shape)
        data_lhs = data_lhs.float().to(device)
        data_rhs = data_rhs.float().to(device)
        data_output = data_output.float().to(device)
        data_x_lens = data_x_lens.int().to(device)
        output, factors = best_model(x_lhs = data_lhs, x_rhs = data_rhs)

        data_mask = torch.arange(data_lhs.shape[2])[None, :].to(device) < data_x_lens[:, None]
        output_masked = torch.masked_select(output,data_mask)
        data_output_masked = torch.masked_select(data_output,data_mask)
        assert (len(output_masked) == torch.sum(data_x_lens))

        indices = [0]+list(torch.cumsum(data_x_lens,axis=0).cpu().numpy())

        for k,(i,j) in enumerate(zip(indices[:-1],indices[1:])):
            output_lst.append(pd.Series(output_masked[i:j].cpu().detach().numpy(),index=data_assets[k]))
            data_lst.append(pd.Series(data_output_masked[i:j].cpu().detach().numpy(),index=data_assets[k]))
            factors_lst.append(pd.Series(factors[k].squeeze().cpu().detach().numpy()))

    gc.collect()
    torch.cuda.empty_cache()
    
    output_df = pd.DataFrame(output_lst,index=extended_valid_data.output_date_idx).iloc[n_window:]
    data_output_df = pd.DataFrame(data_lst,index=extended_valid_data.output_date_idx).iloc[n_window:]
    factors_df = pd.DataFrame(factors_lst,index=extended_valid_data.output_date_idx)
    
    predicted_factors = factors_df.rolling(n_window).mean()
    factors_tensor = torch.tensor(predicted_factors.iloc[n_window:].to_numpy()).unsqueeze(2).float().to(device)
    
    
    pred_output_lst = []
    for i, (data_lhs, _, _, data_x_lens, data_assets) in tqdm(enumerate(valid_loader),total=len(valid_loader)):
        data_lhs = data_lhs.float().to(device)
        data_x_lens = data_x_lens.int().to(device)
        output= best_model.forward_given_factors(x_lhs = data_lhs, factors = factors_tensor[i*batch_size:(i+1)*batch_size])

        data_mask = torch.arange(data_lhs.shape[2])[None, :].to(device) < data_x_lens[:, None]
        output_masked = torch.masked_select(output,data_mask)
        assert (len(output_masked) == torch.sum(data_x_lens))

        indices = [0]+list(torch.cumsum(data_x_lens,axis=0).cpu().numpy())

        for k,(i,j) in enumerate(zip(indices[:-1],indices[1:])):
            pred_output_lst.append(pd.Series(output_masked[i:j].cpu().detach().numpy(),index=data_assets[k]))
    tm1_predicted_returns = pd.DataFrame(pred_output_lst,index=predicted_factors.iloc[n_window:].index)
    predicted_returns = tm1_predicted_returns.shift(1).iloc[1:]
    
    gc.collect()
    torch.cuda.empty_cache()
    
    log_dict = {}
    
#     log_dict['predicted_returns'] = predicted_returns
    
    # Predictive Stats:
    # Spearman Corr
    corr = predicted_returns.corrwith(returns_df.loc[predicted_returns.index,predicted_returns.columns],axis=1,method='spearman')
    log_dict['pred_spearman_corr'] = corr.mean()
    # R2
    numer = (returns_df.loc[predicted_returns.index,predicted_returns.columns]-predicted_returns)
    numer_mask = (~numer.isna()).astype(int)
    denom = returns_df.loc[predicted_returns.index,predicted_returns.columns] * numer_mask
    log_dict['pred_r2'] = (1 - ((numer**2).sum(axis=1))/(denom**2).sum(axis=1)).mean()
    
    # Attribution Stats:
    # Spearman Corr
    corr = output_df.corrwith(returns_df.loc[output_df.index,output_df.columns],axis=1,method='spearman')
    log_dict['total_spearman_corr'] = corr.mean()
    # R2
    numer = (returns_df.loc[output_df.index,output_df.columns]-output_df)
    numer_mask = (~numer.isna()).astype(int)
    denom = returns_df.loc[output_df.index,output_df.columns] * numer_mask
    log_dict['total_r2'] = (1 - ((numer**2).sum(axis=1))/(denom**2).sum(axis=1)).mean()
    
    # Trading Stats:
    train_pred_rank = predicted_returns.rank(pct=True,axis=1,numeric_only=True,ascending=True)
    long_port = ((train_pred_rank >= 0.9) & (train_pred_rank < 1)).astype(int)
    long_port = long_port.divide(long_port.sum(axis=1),axis=0)
    long_port_returns = (long_port*returns_df.loc[long_port.index]).sum(axis=1)
    short_port = (train_pred_rank <= 0.1).astype(int)
    short_port = short_port.divide(short_port.sum(axis=1),axis=0)
    short_port_returns = (short_port*returns_df.loc[short_port.index]).sum(axis=1)
    comb_port = long_port - short_port
    comb_port_returns = (comb_port*returns_df.loc[comb_port.index]).sum(axis=1)
    log_dict['long_short_SR'] = (comb_port_returns.mean()/comb_port_returns.std())*np.sqrt(12)
    log_dict['long_short_returns'] = comb_port_returns.mean()*12
    log_dict['long_short_SD'] = comb_port_returns.std()*np.sqrt(12)
    
    log_dict['long_SR'] = (long_port_returns.mean()/long_port_returns.std())*np.sqrt(12)
    log_dict['long_returns'] = long_port_returns.mean()*12
    log_dict['long_SD'] = long_port_returns.std()*np.sqrt(12)
    
    log_dict['short_SR'] = (short_port_returns.mean()/short_port_returns.std())*np.sqrt(12)
    log_dict['short_returns'] = short_port_returns.mean()*12
    log_dict['short_SD'] = short_port_returns.std()*np.sqrt(12)
    
    return log_dict

In [12]:
fixed_args = {
#     'n_factors' : 5,
#     'n_hidden_lhs':[16,8],
    'n_hidden_rhs':[],
    'batch_norm_lhs' : True,
    'batch_norm_rhs' : False,
#     'rhs_activation': True,
    'dropout_p' : 0.0,
    'max_norm' : 1,
    'bn_momentum': 0.01,
    'bn_track_running_stats': True,
    'initialization' : False,
    'rhs_input_assets' : False,
    'lr' : 5e-4,
    'gamma': 1/4,
    'l2_lambda': 0,
    'l1_lambda': 4e-3,
    'batch_size': 32,
    'log_interval': 1,
    'epoch': 60,
    'patience': 6
}

search_args = {
    'n_factors': [2,4,6],
#     'n_factors': [4],
    'n_hidden_lhs': [(24,),(16,8),(16,8,4)],
#     'n_hidden_lhs': [(16,8)],
    'rhs_activation': [False]
}

# fixed_args = {
# #     'n_factors' : 5,
# #     'n_hidden_lhs':[16,8],
#     'n_hidden_rhs':[],
#     'batch_norm_lhs' : True,
#     'batch_norm_rhs' : False,
#     'rhs_activation': True,
#     'dropout_p' : 0.0,
#     'max_norm' : 1,
#     'bn_momentum': 0.01,
#     'bn_track_running_stats': True,
#     'initialization' : False,
# #     'rhs_input_assets' : False,
#     'lr' : 5e-4,
#     'gamma': 1/4,
#     'l2_lambda': 0,
#     'l1_lambda': 4e-3,
#     'batch_size': 32,
#     'log_interval': 1,
#     'epoch': 3,
#     'patience': 6
# }

# search_args = {
#     'n_factors': [2],
#     'n_hidden_lhs': [(8,)],
#     'rhs_input_assets': [True]
# }

search_keys = list(search_args.keys())
search_values = list(search_args.values())
from itertools import product
search_values = list(product(*search_values))


In [13]:
search_values

[(2, (24,), False),
 (2, (16, 8), False),
 (2, (16, 8, 4), False),
 (4, (24,), False),
 (4, (16, 8), False),
 (4, (16, 8, 4), False),
 (6, (24,), False),
 (6, (16, 8), False),
 (6, (16, 8, 4), False)]

In [14]:
res_dict = {}

with open('res_dict_midterm_corrected.pickle', 'rb') as handle:
    res_dict = pickle.load(handle)

for i,tpl in enumerate(search_values):
    print(f"Params {i}:", tpl)
    if i < len(res_dict):
        continue
    args = fixed_args.copy()
    for k,v in zip(search_keys,tpl):
        args[k] = v
        
    str_tpl = [str(tpl[0]),"_".join(list(map(str,tpl[1]))),"activation" if tpl[2] else "noactivation"]

    log_dict = run(args,n_alphas=29, full_df_lst=full_df_lst, returns_df=returns_df, date_idx=date_idx, 
                   train_start=48, train_end=360, valid_start=360, valid_end=420, 
                   n_window=36, batch_size=16,
                    checkpoint_name=f'midterm_corrected_layers{str_tpl[0]}_hidden_{str_tpl[1]}_{str_tpl[2]}'
#                    checkpoint_name=f'midterm_corrected/layers{str_tpl[0]}_hidden_{str_tpl[1]}_{str_tpl[2]}'
                  )
    
    gc.collect()
    torch.cuda.empty_cache()
    
    res_dict[tpl] = log_dict
    
    with open('res_dict_midterm_corrected.pickle', 'wb') as handle:
        pickle.dump(res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    !cp * ../efs/project

Params 0: (2, (24,), False)
Params 1: (2, (16, 8), False)
Params 2: (2, (16, 8, 4), False)
Params 3: (4, (24,), False)
Params 4: (4, (16, 8), False)
Params 5: (4, (16, 8, 4), False)
Params 6: (6, (24,), False)
ConditionalAutoEncoder(
  (lhs): Sequential(
    (0): LayerNorm((29,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=29, out_features=24, bias=True)
    (2): SELU()
    (3): Linear(in_features=24, out_features=6, bias=True)
  )
  (rhs): Sequential(
    (0): Linear(in_features=30, out_features=6, bias=True)
  )
)
   28.31 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
    4.03 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
Pre-Train Train MSE: 9.99128
Pre-Train Valid MSE: 0.04731148


Epoch 1, total_loss=6.2788, reg_loss = 0.5139
   55.19 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 1, max_norm=1)
   26.04 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
    3.86 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
Train MSE: 3.1923149
Valid MSE: 0.04804948
Next Learning Rate: 0.0005
[9.99128]
[0.04731148]


Epoch 2, total_loss=2.7231, reg_loss = 0.5079
   54.07 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 2, max_norm=1)
   26.04 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
    3.73 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
Train MSE: 1.3553666
Valid MSE: 0.047690954
Next Learning Rate: 0.0005
[9.99128, 3.1923149]
[0.04731148, 0.04804948]


Epoch 3, total_loss=1.5458, reg_loss = 0.5012
   54.85 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 3, max_norm=1)
   26.65 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
    3.76 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
Train MSE: 0.78882
Valid MSE: 0.046349537
Next Learning Rate: 0.0005
[9.99128, 3.1923149, 1.3553666]
[0.04731148, 0.04804948, 0.047690954]


Epoch 4, total_loss=1.1489, reg_loss = 0.4944
   54.94 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 4, max_norm=1)
   26.58 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
    3.71 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
Train MSE: 0.5528462
Valid MSE: 0.0445589
Next Learning Rate: 0.0005
[9.99128, 3.1923149, 1.3553666, 0.78882]
[0.04731148, 0.04804948, 0.047690954, 0.046349537]


...

Epoch 57, total_loss=0.4929, reg_loss = 0.3147
  110.33 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 57, max_norm=1)
   47.20 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
    6.28 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
Train MSE: 0.180677
Valid MSE: 0.034155842
Next Learning Rate: 0.000125
[2.3782287, 0.7950086, 0.35786057, 0.2447466, 0.2167174, 0.20529981, 0.19954538, 0.19615844, 0.1937069, 0.19180797, 0.19013254, 0.18884797, 0.18788813, 0.18694536, 0.18621361, 0.1854996, 0.18498105, 0.184461, 0.18402763, 0.183599, 0.18330263, 0.18300827, 0.18277481, 0.18251938, 0.18233174, 0.18214868, 0.1819795, 0.18187442, 0.18178234, 0.18165587, 0.18158476, 0.18146335, 0.18140522, 0.18133263, 0.18129501, 0.18125676, 0.18122332, 0.18117924, 0.18114445, 0.1811184, 0.18108326, 0.181

Epoch 58, total_loss=0.4857, reg_loss = 0.3135
  110.25 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 58, max_norm=1)
   47.16 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
    6.27 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
Train MSE: 0.18066666
Valid MSE: 0.03415574
Next Learning Rate: 0.000125
[2.3782287, 0.7950086, 0.35786057, 0.2447466, 0.2167174, 0.20529981, 0.19954538, 0.19615844, 0.1937069, 0.19180797, 0.19013254, 0.18884797, 0.18788813, 0.18694536, 0.18621361, 0.1854996, 0.18498105, 0.184461, 0.18402763, 0.183599, 0.18330263, 0.18300827, 0.18277481, 0.18251938, 0.18233174, 0.18214868, 0.1819795, 0.18187442, 0.18178234, 0.18165587, 0.18158476, 0.18146335, 0.18140522, 0.18133263, 0.18129501, 0.18125676, 0.18122332, 0.18117924, 0.18114445, 0.1811184, 0.18108326, 0.18

Epoch 59, total_loss=0.4848, reg_loss = 0.3124
  110.88 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 59, max_norm=1)
   47.43 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
    6.37 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
Train MSE: 0.1806464
Valid MSE: 0.034154505
Next Learning Rate: 0.000125
[2.3782287, 0.7950086, 0.35786057, 0.2447466, 0.2167174, 0.20529981, 0.19954538, 0.19615844, 0.1937069, 0.19180797, 0.19013254, 0.18884797, 0.18788813, 0.18694536, 0.18621361, 0.1854996, 0.18498105, 0.184461, 0.18402763, 0.183599, 0.18330263, 0.18300827, 0.18277481, 0.18251938, 0.18233174, 0.18214868, 0.1819795, 0.18187442, 0.18178234, 0.18165587, 0.18158476, 0.18146335, 0.18140522, 0.18133263, 0.18129501, 0.18125676, 0.18122332, 0.18117924, 0.18114445, 0.1811184, 0.18108326, 0.18

Epoch 60, total_loss=0.4878, reg_loss = 0.3113
  111.27 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 60, max_norm=1)
   46.63 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
    6.28 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cuda, <torch.utils.data.data..., MSELoss())
Train MSE: 0.18063319
Valid MSE: 0.034152985
Next Learning Rate: 0.000125
[2.3782287, 0.7950086, 0.35786057, 0.2447466, 0.2167174, 0.20529981, 0.19954538, 0.19615844, 0.1937069, 0.19180797, 0.19013254, 0.18884797, 0.18788813, 0.18694536, 0.18621361, 0.1854996, 0.18498105, 0.184461, 0.18402763, 0.183599, 0.18330263, 0.18300827, 0.18277481, 0.18251938, 0.18233174, 0.18214868, 0.1819795, 0.18187442, 0.18178234, 0.18165587, 0.18158476, 0.18146335, 0.18140522, 0.18133263, 0.18129501, 0.18125676, 0.18122332, 0.18117924, 0.18114445, 0.1811184, 0.18108326, 0.1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.76s/it]


In [ ]:
res_dict